# Lecture 4: Password CPA Attack - Introduction

In this example we want to improve the password check again to be resistant against the attack from the last tutorial.

## Improving the code

Let's first recap the password checking loop from the last lecture:
```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != passwd[i])
    {
        password_wrong = 1;
    }
}
```

The differences attack discussed in the last example worked because of the different power consumption when executing the code inside the if clause. This is addressed by the following code.

```c
uint8_t password_wrong = 0;
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    password_wrong |= stored_password[i] ^ passwd[i];
}
```

This is an excerpt from `4_password_fixed.c`.

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 1</div>
    
Check that neither SAD nor MAD attack does not work anymore.

</div>


One of the main principle of side channel analysis is:

> The power consumption of a device is proportional to the hamming weight of the data it processes.

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 2</div>

Capture several traces with different input values of different hamming weights. Consider only the _first_ character.

Match the trace points with the assembly instructions of the comparison loop:

- There are points where _all_ traces have the same value.
- There are points where the traces differ.

</div>

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 3</div>

Visualize the proportionality statement above:

1. Capture a trace with an input for each hamming weight.
2. Identify an appropriate trace point where the different hamming weights are "visible".
3. Create a plot "hw(attempt) vs. trace[point]". <br>
   I.e. put the values hw(attempt) on the x-axis and the value of the corresponding trace at the proper point on the y-axis.

How does the plot look like on "wrong" positions?

</div>

In [ ]:
import numpy as np

scope.default_setup()
scope.adc.samples = 500

def capture(attempt, count=1):
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])
    traces = []
    for _ in range(count):
        scope.arm()
        target.simpleserial_write(0x01, attempt + b'\x00' * (8 - len(attempt)))
        result = target.simpleserial_read(0x01, 1)
        traces.append(util.capture())
    return np.mean(np.array(traces), axis=0), not bool(result[0])
